In [2]:
# loads a board for testing purposes
import random, math, copy

def getBoard(difficulty, boardNumber=0):
    def readFile(path):
        with open(path, "rt") as f:
            return f.read()

    board = [['0'] * 9 for _ in range(9)]
    difficultyToRange = {'easy': (1, 50),
                        'medium': (1, 50),
                        'hard': (1, 50),
                        'expert': (1, 25),
                        'evil': (1, 25)}
    if boardNumber == 0:
        boardNumber = random.randint(*difficultyToRange[difficulty])
    path = f'boards/boards/{difficulty}-{str(boardNumber).zfill(2)}.png.txt'
    boardText = readFile(path)
    i = 0
    for line in boardText.splitlines():
        j = 0
        for v in line.split():
            board[i][j] = v
            j += 1
        i += 1
    return board

In [4]:
#this cell contains most of the helper functions
#first backtracker, pretty basic
def solve(board):
    if isSolved(board): return board
    board = copy.deepcopy(board)
    legals = getLegals(board)
    row, col = leastLegals(legals)
    for num in legals[row][col]:
        board[row][col] = num
        result = solve(board)
        if result != None: return result
    return None

def leastLegals(legals):
    minRow, minCol = 0, 0
    for row in range(9):
        for col in range(9):
            if len(legals[row][col]) == 0: continue
            elif len(legals[row][col]) == 1: return row, col
            if (len(legals[minRow][minCol]) == 0 or
                len(legals[row][col]) < len(legals[minRow][minCol])):
                minRow, minCol = row, col
    return minRow, minCol

def getLegals(board):
    legals = [[set()] * 9 for _ in range(9)]
    for row in range(9):
        for col in range(9):
            if board[row][col] == '0':
                legals[row][col] = ({'1', '2', '3', '4', '5', '6', '7', '8', '9'}
                                    - getRegion(board, row, col))
    return legals

def isSolved(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == '0': return False
    return True

def getRow(board, row):
    return board[row]

def getCol(board, col):
    return [board[row][col] for row in range(9)]

def getBlock(board, row, col, flatten=True):
    startRow, startCol = 3 * math.floor(row / 3), 3 * math.floor(col / 3)
    block = [[None] * 3 for _ in range(3)]
    for drow in range(3):
        for dcol in range(3):
            block[drow][dcol] = board[startRow + drow][startCol + dcol]
    if flatten: return block[0] + block[1] + block[2]
    else: return block

def getRegion(board, row, col):
    return set(getRow(board, row)) | set(getCol(board, col)) | set(getBlock(board, row, col))

In [5]:
#this cell is a few of the solvers i tested
from collections import deque

#original solve function
def solve(board):
    if isSolved(board): return board
    board = copy.deepcopy(board)
    legals = getLegals(board)
    row, col = leastLegals(legals)
    for num in legals[row][col]:
        board[row][col] = num
        result = solve(board)
        if result != None: return result
    return None

#current version, does modifications in place 
def solve2(board):
    legals = getLegals(board)

    #function assumes you input the next move it should take
    def recurse(board, lastRow, lastCol, move):
        if isSolved(board): return board
        #given the previous move, legals are updated
        updateLegals(legals, lastRow, lastCol, move)
        row, col = leastLegals(legals)
        #if leastLegals returns an empty set then there are no legal moves
        if legals[row][col] == set(): return None
        for num in copy.copy(legals[row][col]):
            board[row][col] = num
            result = recurse(board, row, col, num)
            if result != None: return result
            board[row][col] = '0'
            #undoes legals (in place)
            undoUpdate(legals, row, col, board)
        return None
    
    return recurse(copy.deepcopy(board), 0, 0, 0)

#helper function that updates only the legals that change based
#on the last move, also does it in place so its faster
def updateLegals(legals, lastRow, lastCol, n):
    #starting row and col for block
    row, col = 3 * math.floor(lastRow / 3), 3 * math.floor(lastCol / 3)
    for i in range(9):
        legals[lastRow][i].discard(n) #row
        legals[i][lastCol].discard(n) #col
        drow, dcol = i // 3, i % 3
        legals[row + drow][col + dcol].discard(n) #block
    legals[lastRow][lastCol] = set()
#undoes legal updates when the move didn't work out
def undoUpdate(legals, lastRow, lastCol, board):
    startRow, startCol = 3 * math.floor(lastRow / 3), 3 * math.floor(lastCol / 3)
    allLegals = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
    for i in range(9):
        drow, dcol = i // 3, i % 3
        for row, col in [(lastRow, i), (i, lastCol), (startRow + drow, startCol + dcol)]:
            if board[row][col] != '0': legals[row][col] = set()
            else: legals[row][col] = allLegals - getRegion(board, row, col)

#similar to original solving but with a stack rather than recursion
#didn't work out well because there was lots of deep copies;
#also tried implementing this with a deque which didn't help much
def solve3(board):
    legals = getLegals(board)
    row, col = leastLegals(legals)

    stack = list()
    stack.append([copy.deepcopy(board), row, col, legals[row][col]])

    while stack:
        board, row, col = stack[-1][:-1]
        move = stack[-1][-1].pop()
        if stack[-1][-1] == set(): stack.pop()
        board[row][col] = move

        if isSolved(board): return board #done if done

        #else keep lookin
        legals = getLegals(board)
        nextRow, nextCol = leastLegals(legals)
        if legals[nextRow][nextCol] != set():
            stack.append([copy.deepcopy(board), nextRow,
                          nextCol, legals[nextRow][nextCol]])


In [ ]:
import timeit
d = 'evil'
board = getBoard(d)
#makeing sure solutions were valid
%timeit assert(solve(board) != None)
%timeit assert(solve2(board) != None)
%timeit assert(solve3(board) != None)
#testing solutions on all evil boards
%timeit -n 1 [solve(getBoard(d, i + 1)) for i in range(25)]
%timeit -n 1 [solve2(getBoard(d, i + 1)) for i in range(25)]
%timeit -n 1 [solve3(getBoard(d, i + 1)) for i in range(25)]

In [34]:
#timing different ways of getting legals
board = getBoard('evil')
# assert(solve2(board) == solve(board))
legals = getLegals(board)
%timeit getLegals(board)
#%timeit getLegals2(board) #no longer exists because it sucked
%timeit updateLegals(legals, 0, 0, '3')

201 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
193 µs ± 9.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.94 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [257]:
# this was for refactoring code so that you can resize the app window
from fractions import Fraction
width = 120
height = 40
f1 = Fraction(width, 1000)
f2 = Fraction(height, 800)
if f1.numerator == 1: s1 = f"app.width/{f1.denominator}"
else: s1 = f"app.width*{f1.numerator}/{f1.denominator}"
if f2.numerator == 1: s2 = f"app.height/{f2.denominator}"
else: s2 = f"app.height*{f2.numerator}/{f2.denominator}"
print(f"{s1}, {s2}")

app.width*3/25, app.height/20
